In [3]:
# !pip install scikit-surprisea

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler

matplotlib.rcParams['axes.prop_cycle'] = cycler(color=['#007efd', '#FFC000', '#303030'])

In [2]:
df_ratings = pd.read_parquet('ratings.parquet')
df_ratings.tail()

,user_id,item_id,rating,timestamp
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [4]:
df_items = pd.read_parquet('/home/valdir/Documentos/CURSO_DNC/novo-curso-dnc/curso-dnc/MODULO-10/EXEMPLO/Anexos_item-knn/movies.parquet')
df_items.set_index('item_id', inplace=True)
df_items.tail()

,title,genres
item_id,,
3948,Meet the Parents (2000),Comedy
3949,Requiem for a Dream (2000),Drama
3950,Tigerland (2000),Drama
3951,Two Family House (2000),Drama
3952,"Contender, The (2000)",Drama|Thriller


In [5]:
train_size = 0.8
# Ordenar por timestamp
df_ratings = df_ratings.sort_values(by='timestamp', ascending=True)

In [6]:
# Definir treino e valid sets
df_train_set, df_valid_set = np.split(df_ratings,[int(train_size*df_ratings.shape[0])])

In [7]:
# Redefinindo os nomes para user_id e itens_id

df_train_set = df_train_set.rename({'user_id':'userID', 'item_id':'itemID'}, axis=1)
df_valid_set = df_valid_set.rename({'user_id':'userID', 'item_id':'itemID'}, axis=1)

In [8]:
df_train_set

,userID,itemID,rating,timestamp
1000138,6040,858,4,956703932
1000153,6040,2384,4,956703954
999873,6040,593,5,956703954
1000007,6040,1961,4,956703977
1000192,6040,2019,5,956703977
...,...,...,...,...
314102,1875,802,4,975768718
314151,1875,892,4,975768719
314073,1875,440,4,975768738
314225,1875,509,4,975768738


In [12]:
from surprise import Dataset, Reader
from surprise import KNNBasic

def convert_trin_valida_sets(df_train_set:pd.DataFrame, df_valid_set:pd.DataFrame):
    reader = Reader(rating_scale=(1, 5))
    train_set = (
        Dataset.load_from_df(df_train_set[['userID', 'itemID', 'rating']], reader)
        .build_full_trainset()
    )
    valid_set = (
        Dataset.load_from_df(df_valid_set[['userID', 'itemID', 'rating']], reader)
        .build_full_trainset()
        .build_testset()
    )
    return train_set, valid_set

train_set, valid_set = convert_trin_valida_sets(df_train_set, df_valid_set)

In [13]:
train_set

In [15]:
""" # Mais Hiperparametro, para escolher o melhor
# Usar a biblioteca Hyperopt: hyperopt.github.io/hyperopt
from surprise import KNNWithMeans

sim_options = {
    "name":"pearson_baseline",
    "user_based":False # computa similaridades entre ítens
}
model = KNNWithMeans(k=40, sim_options=sim_options,verbose=True)
model """
from surprise import KNNWithMeans
sim_options = {
    "name": "pearson_baseline",
    "user_based": False,  # compute similarities between items
}
model = KNNWithMeans(k=40, sim_options=sim_options, verbose=True)
model

In [16]:
model.fit(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [17]:
user_id = df_valid_set['userID'].sample().tolist()[0]
item_id = df_valid_set['itemID'].sample().tolist()[0]
model.predict(uid=user_id,iid=item_id)

Prediction(uid=1447, iid=196, r_ui=None, est=3.1467219328406175, details={'actual_k': 2, 'was_impossible': False})

In [18]:
df_valid_set['prediction'] = df_valid_set.apply(
    lambda x: model.predict(uid=x['userID'], iid=x['itemID']).est,
    axis=1
)

In [20]:
df_valid_set.tail()

,userID,itemID,rating,timestamp,prediction
825793,4958,2399,1,1046454338,2.561691
825438,4958,1407,5,1046454443,3.552168
825724,4958,3264,4,1046454548,3.270295
825731,4958,2634,3,1046454548,2.754026
825603,4958,1924,4,1046454590,2.584696


In [ ]:
# import interact
""" from ipywidgets import interact
@interact(user_id=df_valid_set['userID'].unique()) """
def recommend_n_items(model, user_id,item_ids:np.array, n=20):
  df_predictions = pd.DataFrame(columns=['item_id', 'score'])
  for item_id in item_ids:
    prediction = model.predict(uid=user_id, iid=item_id).est
    df_predictions.loc[df_predictions.shape[0]] = [item_id, prediction]
  
  user_predictions = (
      df_predictions
      .sort_values(by='score', ascending=False)
      .head(n)
      .set_index('item_id')
  )
  return user_predictions

user_id = 4958
recommendable_items = df_items.index.values
recommend_n_items(model, user_id,recommendable_items, n=5)